In [1]:
import pymupdf4llm
from markdown_chunker import MarkdownChunkingStrategy

Consider using the pymupdf_layout package for a greatly improved page layout analysis.


In [2]:
class Chunk:
    def __init__(self, chunk: str, page: int):
         self.chunk = chunk
         self.page = page

In [3]:
class Chunker:
    def __init__(self): 
        self.strategy = MarkdownChunkingStrategy(min_chunk_len=512,    # Minimum chunk size (default: 512)
                                                soft_max_len=800,       # Preferred maximum chunk size (default: 1024)
                                                hard_max_len=1024,      # Absolute maximum chunk size (default: 2048)
                                                detect_headers_footers=False,   # Detect and remove repeating headers/footers
                                                remove_duplicates=False,        # Remove duplicate chunks
                                                add_metadata=False,             # Add metadata in each chunk as YAML front matter,
                                                parallel_processing=True,
                                                max_workers=4
                                                )

    def chunk_document(self, data_file, page_to=None) -> list[Chunk]:
        if data_file.lower().endswith(".md"):
            with open(data_file, "r", encoding="utf-8") as f:
                md_text = f.read()
        else:
            # docling is not able to serialize links so we use pymupdf4llm
            if page_to:
                md_text = pymupdf4llm.to_markdown(doc=data_file, pages=(0, page_to-1))
            else:
                md_text = pymupdf4llm.to_markdown(doc=data_file)
        results = []
        chunks = self.strategy.chunk_markdown(md_text)
        for i, chunk in enumerate(chunks):
            results.append(Chunk(chunk=str(chunk), page=-1)) # page ignored for now
        return results

In [4]:
pdf_path = "../../data/raw/kalender-akademik/2024-2025.pdf"

In [5]:
chunker = Chunker()
chunks = chunker.chunk_document(pdf_path)

print(f"Total chunk: {len(chunks)}")

Total chunk: 18


In [6]:
for i, chunk in enumerate(chunks):
    print("-----")
    print(f"Chunk {i}")
    print(chunk.chunk)

-----
Chunk 0
**SURAT KEPUTUSAN**
**SURAT KEPUTUSAN**


**REKTOR UNIVERSITAS PERTAMINA**

**NOMOR : 0103/UPER-R/SK/HK.01/VI/2024**

**TENTANG**

**KALENDER AKADEMIK UNIVERSITAS PERTAMINA TAHUN AKADEMIK 2024/2025**


DENGAN RAHMAT TUHAN YANG MAHA ESA

**REKTOR UNIVERSITAS PERTAMINA**



Menimbang :


Mengingat :



a. bahwa untuk menjamin pelaksanaan kegiatan pembelajaran Tahun Akademik


2024/2025 di Universitas Pertamina dapat dilaksanakan dengan lancar dan tertib,


perlu ditetapkan Kalender Akademik Universitas Pertamina Tahun Akademik


2024/2025;


b. bahwa sehubungan dengan butir a dan b di atas, perlu ditetapkan dalam Surat


Keputusan Rektor Universitas Pertamina.


-----
Chunk 1
1. Undang-Undang Republik Indonesia Nomor 12 Tahun 2012 tentang Pendidikan


Tinggi;


2. Peraturan Pemerintah Republik Indonesia Nomor 4 Tahun 2014 tentang


Penyelenggaraan Pendidikan Tinggi dan Pengelolaan Perguruan Tinggi;


3. Peraturan Menteri Pendidikan, Kebudayaan, Riset, dan Teknologi Nomor 53

In [7]:
import pickle

# --- MENYIMPAN ---
# Ambil teks dari chunks
texts_to_save = [c.chunk for c in chunks]

with open("2024_akademik_chunks_data.pkl", "wb") as f:
    pickle.dump(texts_to_save, f)
print("Data chunks berhasil disimpan ke pkl.")

# --- MEMBACA (Di script embedding) ---
# with open("chunks_data.pkl", "rb") as f:
#     loaded_texts = pickle.load(f)

# Sekarang loaded_texts bisa langsung masuk ke embedding
# client.encode_documents(loaded_texts)

Data chunks berhasil disimpan ke pkl.


In [8]:
import json

with open("2024_akademik_chunks_data.json", "w", encoding="utf-8") as f:
    json.dump(texts_to_save, f, ensure_ascii=False, indent=4)
print("Data chunks berhasil disimpan ke json.")

Data chunks berhasil disimpan ke json.
